# Plan and Execute for Deep Web Search 

- This code is designed to plan and execute a search on the deep web using a specified search engine.


In [9]:
import sys
import os
from openai import AzureOpenAI
import requests
import json
from urllib.parse import urljoin
from langchain_core.prompts import PromptTemplate
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from langchain_core.prompts import load_prompt
import sys
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from utils.search_utils import web_search, url_search, extract_contexts_async
from IPython.display import Markdown, display
from datetime import datetime
import pytz
import time
import os
from dotenv import load_dotenv
load_dotenv(override=True) 
import logging

logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Get credentials from environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 10))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "gpt-4o-mini")


client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

current_dir = os.getcwd()

def rewrite_query_for_search(query, client: AzureOpenAI):    
        
        # Customize your prompt for query rewriting
        QUERY_REWRITE_PROMPT = """
          You are an expert in rewriting user queries to improve search results. 
          Here is the user's original query:
          {user_query}
          
          <<Format>>
          The response should be in JSON Object format with the following keys
          {{
          "search_query": "검색용 재작성된 질문"
          }}
        """
        response = client.chat.completions.create(
            model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT.format(
                  user_query=query)},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )

        print("Rewritten query for search:", response.choices[0].message.content.strip())
        
        return json.loads(response.choices[0].message.content.strip())

    
  
def plan_query_for_search(query, client: AzureOpenAI):

  # TODO Customize your prompt
  PLANNER_PROMPT = """
    You are an expert in planning  to gather relevant information.
    The user has provided the following query:
    {query}
    
    <<Format>>
    The response should be in JSON Object format with the following keys:
    {{
    "search_queries": ["{query}"],
    }}

  """

  response = client.chat.completions.create(
  model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
  messages=[
    {"role": "system", "content": PLANNER_PROMPT.format(query=query)},
    {"role": "user", "content": query}
  ],
  temperature=0.8,
  max_tokens=300,
  response_format= {"type": "json_object"},
  )

  print("Planned query for search:", response.choices[0].message.content.strip())

  return json.loads(response.choices[0].message.content.strip())


In [12]:
# 웹 검색 결과를 활용해 LLM 답변을 생성하는 비동기 함수
async def process_deep_search(max_result_count=3, input_query=None, web_search_mode=None, product_name=None):
    
    start_time = time.time()
    print(f"Original Input: {input_query}")

    # 검색 모드가 지정되지 않으면 환경변수
    if web_search_mode is None:
        web_search_mode = os.getenv("WEB_SEARCH_MODE", "google").lower()
    print(f"############## Web Search Mode: {web_search_mode}")
    
    # query rewrite (검색용/LLM용)
    query_rewrite = rewrite_query_for_search(input_query, client)
    
    # plan deep query
    search_plan_json = plan_query_for_search(query_rewrite["search_query"], client)
    print(f"Planned Search Queries: {search_plan_json}")
    search_queries = search_plan_json.get("search_queries", [])
    all_contexts = []
    
    for i, query in enumerate(search_queries):
        print(f"Processing query {i+1}: {query}")
        search_results = url_search(query, max_result_count, web_search_mode=web_search_mode, product_name=product_name)
        
        print(f"Search Results: {len(search_results)} results found for query '{query}'")
        
        print("Analyze search results...")        
        if search_results:
            url_snippet_tuples = [(r["link"], r["snippet"]) for r in search_results]
            contexts = await extract_contexts_async(url_snippet_tuples=url_snippet_tuples)
            
            formatted_contexts = [
                f"[search keyword: {query}]\n{context}"
                for context in contexts
            ]
            
            all_contexts.extend(formatted_contexts)
        
    
    
    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")
        
    # Get the current working directory for this notebook
    GENERATE_PROMPT = """
    <<Customize your prompt here>>
    You are an expert in generating responses based on search results. Your task is to generate a response using the provided search results and the user's query.
    Here are the search results:
    {contexts}
    Here is the user's query:
    {user_query}
    """
    
    print("Generate response...")            
    
    answer_messages = [
        {"role": "system", "content": GENERATE_PROMPT.format(
            product_name=product_name,
            date=current_date,
            contexts=all_contexts,
            user_query=query_rewrite['search_query'],
        )},
        {"role": "user", "content": query_rewrite['search_query']}
    ]
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=answer_messages,
        top_p=0.9,
        max_tokens=1500
    )
    display(Markdown(response.choices[0].message.content))
    
    end_time = time.time()
    print(f"elapsed time: {end_time - start_time:.2f} seconds\n\n")


In [14]:
RESULTS_COUNT = 5

inputs = [
    "티뷰론과 아이오닉5를 비교해줘"
]

web_search_mode = "bing"

for input in inputs:
    print(f"Bing Grounding 검색 사용: {input}")
    await process_deep_search(max_result_count=RESULTS_COUNT, input_query=input, web_search_mode=web_search_mode, product_name="현대자동차") 

Bing Grounding 검색 사용: 티뷰론과 아이오닉5를 비교해줘
Original Input: 티뷰론과 아이오닉5를 비교해줘
############## Web Search Mode: bing


Rewritten query for search: {
  "search_query": "티뷰론과 아이오닉5의 비교 및 차이점"
}
Planned query for search: {
    "search_queries": ["티뷰론과 아이오닉5의 비교 및 차이점"]
}
Planned Search Queries: {'search_queries': ['티뷰론과 아이오닉5의 비교 및 차이점']}
Processing query 1: 티뷰론과 아이오닉5의 비교 및 차이점
Search Results: 0 results found for query '티뷰론과 아이오닉5의 비교 및 차이점'
Analyze search results...
Generate response...


티뷰론과 아이오닉5는 현대자동차에서 생산한 서로 다른 유형의 차량으로, 여러 면에서 차이를 보입니다.

1. **차량 유형**:
   - **티뷰론**: 스포츠 쿠페로, 주로 성능과 디자인을 중시하는 소비자들을 위해 제작되었습니다. 
   - **아이오닉5**: 전기 SUV로, 현대의 전기차 라인업의 일환으로 설계되었으며, 실용성과 첨단 기술을 강조합니다.

2. **구동 방식**:
   - **티뷰론**: 일반적으로 가솔린 엔진을 사용하는 차량으로, 다양한 엔진 옵션을 제공하여 다이내믹한 주행 성능을 제공합니다.
   - **아이오닉5**: 전기차로, 배터리 전기 모터를 사용하여 친환경적이며, 전기 충전으로 구동됩니다.

3. **디자인 및 크기**:
   - **티뷰론**: 스포티하고 날렵한 외관을 가지고 있으며, 상대적으로 작은 크기로 젊은층에게 인기가 많았습니다.
   - **아이오닉5**: 미래적인 디자인과 넉넉한 실내 공간을 제공하며, 현대적인 전기차 디자인 언어를 따릅니다.

4. **주행 거리 및 효율성**:
   - **티뷰론**: 가솔린 차량으로 연비는 엔진의 종류에 따라 다르지만, 전기차와 비교하면 효율성이 낮습니다.
   - **아이오닉5**: 뛰어난 전비와 함께 한 번의 충전으로 상당한 주행 거리를 자랑하여, 전기차 사용자에게 실용적인 선택이 됩니다.

5. **기술 및 편의성**:
   - **티뷰론**: 기본적인 편의 장치와 함께 스포티한 주행에 집중한 차량입니다.
   - **아이오닉5**: 최신 기술이 접목된 차량으로, 다양한 안전 및 편의 기능, 그리고 스마트한 인포테인먼트 시스템을 갖추고 있습니다.

결론적으로, 티뷰론은 스포티한 주행을 선호하는 소비자에게 적합하며, 아이오닉5는 전기차의 장점과 실용성을 중시하는 사용자에게 맞는 선택입니다. 각 차량의 목적과 특성에 따라 선택이 달라질 수 있습니다.

elapsed time: 44.71 seconds




# Challenge: Plan and Execute for Deep Web Search 

### Requirements:
1. Please provide detailed answers even short questions.
1. Please provide detailed answers even for complex questions.
1. Please revise the code to get more information from the web search results.